In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from mtflearn.clustering import gmm_lbs
from mtflearn.io import load_image
from stemplot import imshow, colors_from_lbs
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints
from mtflearn.utils import normalize_image, remove_bg
from mtflearn.manifold import ForceGraph8
from sklearn.manifold import TSNE as tsne
from stemplot.colors import color_palette,xy2colors
from skimage.transform import resize
from stemplot.interactive import interactive_clusters
import umap

In [2]:
img = load_image('ADF2_27.dm4')
pts = np.load('Co 3 pts.npy')
lbs = np.load('Co 3 lbs.npy').astype(int)

In [3]:
if img.shape[0] == 2048:
    img = resize(img, (1024, 1024), order=0)
img = normalize_image(img)
# mask1 = np.logical_or(lbs==0, lbs==1)
mask1 = lbs==0
mask2 = np.logical_or(lbs==2, lbs==3)
size = 29

In [4]:
kp = KeyPoints(pts, img, size)
ps = kp.extract_patches(size)
zps = ZPs(n_max=12, size=ps.shape[1])
moments = zps.fit_transform(ps)

### TSNE

In [5]:
tsne_X1 = X1 = (moments.data)[mask1][:, 1:]

In [6]:
tsne_res = tsne(n_components=2).fit_transform(tsne_X1)

In [7]:
tsne_lbs = gmm_lbs(tsne_res,10)

In [8]:
app = interactive_clusters(tsne_res, img, pts[mask1], ps[mask1], lbs=tsne_lbs, s=1)
plt.savefig('tsne.png',transparent=True)

In [18]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = tsne_lbs,cmap = 'tab10',s=7)
ax.axis('off')
plt.savefig('img with tsne color.png',dpi = 700,transparent=True)

In [18]:
fig,ax = plt.subplots(2,5, figsize=(15,5.5))
for i in range(10):
    select_ps = np.mean(ps[mask1][tsne_lbs == i],axis = 0)
    ax[i//5,i%5].imshow(select_ps,cmap= color_palette('C{}'.format(i)))
    ax[i//5,i%5].axis('off')
plt.savefig('tsne ps.png',dpi = 700,transparent=True)

In [19]:
imshow(select_ps,cmap= color_palette('C0'))

### FR

In [19]:
X = moments.select([0]).data
FR_X1 = X[mask1][:, 1:]

In [20]:
fg = ForceGraph8(X = FR_X1,
                 n_neighbors=5,
                 init_mode = 'pca',
                 num_iterations = 100,
                 num_negative_samples=5,
                 force_params1=(0, 2, 1, 1),
                 force_params2=(2, 5, 5, 1),
                 divide=0.5)

from time import time
t0 = time()
FR_res = fg.fit_transform(FR_X1)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [21]:
FR_lbs = gmm_lbs(FR_res,10)

In [44]:
app = interactive_clusters(FR_res, img, pts[mask1], ps[mask1], lbs=FR_lbs, s=1)
plt.savefig('fr.png',transparent=True)

In [45]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = FR_lbs,cmap = 'tab10',s=7)
ax.axis('off')
plt.savefig('img with FR color.png',dpi = 700,transparent=True)

In [22]:
fig,ax = plt.subplots(2,5, figsize=(15,5.5))
for i in range(10):
    select_ps = np.mean(ps[mask1][FR_lbs == i],axis = 0)
    ax[i//5,i%5].imshow(select_ps,cmap= color_palette('C{}'.format(i)))
    ax[i//5,i%5].axis('off')
plt.savefig('FR ps.png',dpi = 700,transparent=True)

### umap

In [6]:
umap_X1 = X1 = (moments.data)[mask1][:, 1:]
umap_res = umap.UMAP(n_components=2).fit_transform(umap_X1)

In [7]:
umap_lbs = gmm_lbs(umap_res,10)

In [8]:
app = interactive_clusters(umap_res, img, pts[mask1], ps[mask1], lbs=umap_lbs, s=1)
plt.savefig('umap.png',transparent=True)

In [9]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = umap_lbs,cmap = 'tab10',s=7)
ax.axis('off')
plt.savefig('img with umap color.png',dpi = 700,transparent=True)

In [13]:
fig,ax = plt.subplots(2,5, figsize=(15,5.5))
for i in range(10):
    select_ps = np.mean(ps[mask1][umap_lbs == i],axis = 0)
    ax[i//5,i%5].imshow(select_ps,cmap= color_palette('C{}'.format(i)))
    ax[i//5,i%5].axis('off')
plt.savefig('umap ps.png',dpi = 700,transparent=True)

### 

In [29]:
def xy2colors_fix_raduis(xy, v=0.9, return_rgb=True):
    # Calculate the angles from xy coordinates
    angles = (np.arctan2(xy[:, 1], xy[:, 0]) + np.pi) / (2 * np.pi)

    # Since we want colors to vary only with angle, we create a constant saturation
    # and use the same value for brightness.
    # The saturation here is set to 1 for maximum color intensity.
    saturation = np.ones_like(angles)  # S = 1 for all points
    value = np.ones_like(angles) * v   # V = v for all points

    # Construct the HSV color representation
    hsv_colors = np.vstack([angles, saturation, value]).T

    # Convert to RGB if required
    if return_rgb:
        return hsv_to_rgb(hsv_colors)
    else:
        return hsv_colors

def xy2colors_fix_angle(xy, v=0.9, return_rgb=True):
    # Compute the radius and normalize it
    radius = np.hypot(xy[:, 0], xy[:, 1])
    max_radius = np.max(radius)
    if max_radius > 0:
        radius = radius / max_radius

    # Since we want colors to vary only with radius, we create a constant hue
    # and use the same value for brightness.
    # The hue here is set based on the normalized radius.
    hue = radius  # H = radius for all points
    saturation = np.ones_like(radius)  # S = 1 for all points
    value = np.ones_like(radius) * v   # V = v for all points

    # Construct the HSV color representation
    hsv_colors = np.vstack([hue, saturation, value]).T

    # Convert to RGB if required
    if return_rgb:
        return hsv_to_rgb(hsv_colors)
    else:
        return hsv_colors

In [10]:
polar = xy2colors(tsne_res, v=0.9, return_rgb=True)

In [20]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.scatter(tsne_res[:,0],tsne_res[:,1],c=polar,s=5)
plt.savefig('tsne polar.png',transparent=True)

In [19]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = polar,s=7)
ax.axis('off')
plt.savefig('img with tsne polar color.png',dpi = 700,transparent=True)

In [24]:
r_polar = xy2colors_fix_raduis(tsne_res,v = 0.9,return_rgb=True)

In [26]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.scatter(tsne_res[:,0],tsne_res[:,1],c=r_polar,s=5)
plt.savefig('tsne polar fix raduis.png',transparent=True)

In [27]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = r_polar,s=7)
ax.axis('off')
plt.savefig('img with tsne polar color fix radius.png',dpi = 700,transparent=True)

In [30]:
a_polar = xy2colors_fix_angle(tsne_res,v = 0.9,return_rgb=True)

In [32]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.scatter(tsne_res[:,0],tsne_res[:,1],c=a_polar,s=5)
plt.savefig('tsne polar fix angle.png',transparent=True)

In [33]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = a_polar,s=7)
ax.axis('off')
plt.savefig('img with tsne polar color fix angle.png',dpi = 700,transparent=True)

In [10]:
def sector_indices(xy,sector_number = 11):
    angles = np.arctan2(xy[:, 1], xy[:, 0])
    sector_boundaries = np.linspace(-np.pi, np.pi, sector_number)
    grouped_indices = []
    for i in range(len(sector_boundaries)-1):
        start = sector_boundaries[i]
        end = sector_boundaries[i+1]
        indices = np.where((angles >= start) & (angles < end))
        grouped_indices.append(indices)
        # grouped_points.append(xy[indices])
    return grouped_indices

def radial_indices(xy,radial_number = 6):
    radius = np.hypot(xy[:, 0], xy[:, 1])
    max_radius = np.max(radius)
    radial_boundaries = np.linspace(0, max_radius, radial_number)
    grouped_indices = []
    for i in range(len(radial_boundaries)-1):
        start = radial_boundaries[i]
        end = radial_boundaries[i+1]
        indices = np.where((radius >= start) & (radius < end))
        grouped_indices.append(indices)
    return grouped_indices

def sector_features(xy,grouped_indices):
    grouped_features = []
    for indices in grouped_indices:
        grouped_features.append(xy[indices])
    return grouped_features

def sector_pts(pts,grouped_indices):
    grouped_pts = []
    for indices in grouped_indices:
        grouped_pts.append(pts[indices])
    return grouped_pts

def sector_ps(ps,grouped_indices,mean = True):
    grouped_ps = []
    for indices in grouped_indices:
        grouped_ps.append(ps[indices])
    new_ps = []
    if mean:
        for g in grouped_ps:
            new_ps.append(np.mean(g,axis = 0))
        return new_ps
    else:
        return grouped_ps


In [9]:
pts = pts[mask1]
ps = ps[mask1]

In [88]:
grouped_indices = sector_indices(tsne_res)
grouped_features = sector_pts(tsne_res,grouped_indices)
grouped_pts = sector_pts(pts,grouped_indices)
grouped_ps = sector_ps(ps,grouped_indices)

In [80]:
tsne_res.shape,pts.shape,ps.shape

((4114, 2), (4114, 2), (4114, 29, 29))

In [12]:
colors = plt.get_cmap('tab10')

In [94]:
for idx,[select_pts,select_features,select_ps] in enumerate(zip(grouped_pts,grouped_features,grouped_ps)):
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    ax[0].imshow(img,cmap = 'gray')
    ax[0].axis('off')
    ax[0].scatter(select_pts[:,0],select_pts[:,1],color = colors(idx),s = 7)
    ax[1].scatter(tsne_res[:,0],tsne_res[:,1],color='black',s = 3)
    ax[1].scatter(select_features[:,0],select_features[:,1],color = colors(idx),s=3)
    ax[2].imshow(select_ps,cmap = 'gray',vmin = select_ps.min(),vmax = select_ps.max()+0.1)
    ax[2].axis('off')
    plt.savefig('sector {}.png'.format(idx),dpi = 700)
    plt.close(fig)

In [18]:
grouped_indices = radial_indices(tsne_res,radial_number = 8)
grouped_features = sector_pts(tsne_res,grouped_indices)
grouped_pts = sector_pts(pts,grouped_indices)
grouped_ps = sector_ps(ps,grouped_indices)

In [16]:
for idx,[select_pts,select_features,select_ps] in enumerate(zip(grouped_pts,grouped_features,grouped_ps)):
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    ax[0].imshow(img,cmap = 'gray')
    ax[0].axis('off')
    ax[0].scatter(select_pts[:,0],select_pts[:,1],color = colors(idx),s = 7)
    ax[1].scatter(tsne_res[:,0],tsne_res[:,1],color='black',s = 3)
    ax[1].scatter(select_features[:,0],select_features[:,1],color = colors(idx),s=3)
    ax[2].imshow(select_ps,cmap = 'gray',vmin = select_ps.min(),vmax = select_ps.max()+0.1)
    ax[2].axis('off')
    plt.savefig('radial {}.png'.format(idx),dpi = 700)
    plt.close(fig)

In [17]:
len(grouped_ps)

7